In [1]:
##################################################################
# Purpose: Create a 3D animation using the published Mopra dataset
# Author:  G. Wong
# Date:    15.10.21
#
# Notes:   This is an update from a previous version from 2015.
#          Suitable for python 3.7 only.
#
#          package name change - image now PIL (Pillow)
#                              - pyfits integrated into astropy.fits
#
##################################################################

In [2]:
# System Checks
import sys
print (sys.version)

3.7.10 | packaged by conda-forge | (default, Oct 13 2021, 20:22:05) [MSC v.1916 64 bit (AMD64)]


In [3]:
#conda activate python3.7   # to be used within jupyter notebook python 3.7

In [4]:
# import libraries
from astropy.io import fits
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D    # used in matplotlib
import numpy as np
import scipy.ndimage as ndimage

from PIL import Image                # used to export the PS files

from mayavi import mlab
from mayavi.mlab import *


In [5]:
# Parameters
reverselut=False

FITSfilename = "G330.5/G330.5-12CO_3sig_vcrop.fits"  #Blues     #The name of the file to read in.
contours = np.arange(6,12,2)		#Specify the contour levels. The numbers are start, end, stepsize
colourscale = "Blues"
colourlimits = [1,10]     #min,max of colourbar
colour = (0,0,1)
print('Contour levels =', contours)

#framesize = [1280,680]   
framesize = [1280,720]  


Contour levels = [ 6  8 10]


In [6]:
# Extract header
fitsfile = fits.open(FITSfilename)
data_cube = fitsfile[0].data
header = fitsfile[0].header
fitsfile.close()

In [7]:
# Extract header infomration
crpix1 = header['CRPIX1']		#reference pixel 
cdelt1 = header['CDELT1']		#step size
crval1 = header['CRVAL1']		#coordinate for reference pixel
ctype1 = header['CTYPE1']

crpix2 = header['CRPIX2']		#latitude
cdelt2 = header['CDELT2']
crval2 = header['CRVAL2']
ctype2 = header['CTYPE2']

crpix3 = header['CRPIX3']		#velocity
cdelt3 = header['CDELT3']
crval3 = header['CRVAL3']	
ctype3 = header['CTYPE3']

bunit = header['BUNIT']

In [8]:
# Data formatting for visualisation, create a blank cube and fill it with the data
print(data_cube[0,0,0], type(data_cube[0,0,0]))
print(np.nanmean(data_cube))
#data_cube = data_cube*1.0 			#sometimes the formatting is wrong, so multiply everything by 1 to fix.
data_cube = data_cube.astype(np.float)

#print data_cube.type
print(data_cube[0,0,0], type(data_cube[0,0,0]))
print(np.nanmean(data_cube))
print('Data cube axes =', ctype3, ctype2, ctype1)
print('Data cube size = ', data_cube.shape)

vel = [((x+1-crpix3)*cdelt3+crval3)/1000 for x in range(data_cube.shape[0])]		#+1 because the FITS pixels are 1 to n, the python indexes are 0 to (n-1)
glat = [(x+1-crpix2)*cdelt2+crval2 for x in range(data_cube.shape[1])]
glon = [(x+1-crpix1)*cdelt1+crval1 for x in range(data_cube.shape[2])]

data_cube = data_cube[::-1,:,:] 			#Reverse the velocity axis, so that positive velocity is further away. Maybe not necessary?
#data_cube = data_cube[39:172,8:126,10:600]		#If you want to cut some edges off the cube, you can specify it here
vel = vel[1:1700]   # ctype3
glat = glat[1:122]  # ctype2
glon = glon[1:122]  # ctype1


vel = vel[::-1]

print('Axes size =', len(vel), len(glat), len(glon))
print('Axes limits =', [vel[0],vel[-1]], [glat[0],glat[-1]], [glon[0],glon[-1]])

print(data_cube[0,0,0], type(data_cube[0,0,0]))
data_cube[np.isnan(data_cube)]= 0 			#set any nan values to zero
print(data_cube[0,0,0], type(data_cube[0,0,0]))

data_cube = ndimage.gaussian_filter(data_cube, sigma=(0.5, 0.5, 0.5), order=0) 			#apply a gaussian filter to smooth the data. sigma = standard deviation for each axis

data_cube=np.transpose(data_cube,(0,2,1))   #sort into velocity, longitude, latitude order. Might depend on your fits file


nan <class 'numpy.float32'>
8.477997
nan <class 'numpy.float64'>


C:\Users\gwong\anaconda3\envs\python3.7\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """


8.478003629037465
Data cube axes = VELO-LSR GLAT-CAR GLON-CAR
Data cube size =  (1701, 122, 122)
Axes size = 1699 121 121
Axes limits = [39.9, -129.9] [-0.493750000000395, 0.506250000000405] [330.9937499999844, 329.9937499999836]
nan <class 'numpy.float64'>
0.0 <class 'numpy.float64'>


In [9]:
# Imaging 

mlab.figure(size=framesize)
scalarfield = mlab.pipeline.scalar_field(data_cube)		#create a scalar field object from the data cube
scalarfield.spacing = [1,1,1]							#stretch out the velocity axis.

vol = mlab.pipeline.volume(scalarfield, vmin=contours[0],vmax=contours[1],color=colour)

mlab.figure(figure = mlab.gcf(),bgcolor=(1,1,1),fgcolor = (0,0,0))

f = mlab.outline()
f.outline_mode='cornered'

vol.module_manager.scalar_lut_manager.data_range = [colourlimits[0], colourlimits[1]]
vol.module_manager.scalar_lut_manager.data_range = [contours[0], contours[-1]]
vol.module_manager.scalar_lut_manager.reverse_lut = reverselut


In [10]:
# Image settings - title

ti = mlab.title('12CO 1-0',height=0.9, size = 0.3)
ti.y_position = 0.9


In [11]:
# Camera orientation - loop to generate multiple images

f = mlab.gcf()

deg = 0
for i in range(1,270):
    view(deg,90,distance=2000,focalpoint='auto')
    mlab.savefig(f'images/{i:04d}_vol.jpeg')
    deg = deg + (i*0.01)

    
# mlab.show()   # good for debugging
